<a href="https://colab.research.google.com/github/Weimw/movielens-project/blob/main/CS224w_Link_Prediction_on_MovieLens(ver1).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
from torch import Tensor
print(torch.__version__)

1.13.1+cu116


In [2]:
# Install required packages.
%%capture
import os
os.environ['TORCH'] = torch.__version__

!pip install torch-scatter -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install torch-sparse -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install pyg-lib -f https://data.pyg.org/whl/nightly/torch-${TORCH}.html
!pip install git+https://github.com/pyg-team/pytorch_geometric.git

In [11]:
# import required modules
import random
from tqdm import tqdm
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

from torch import nn, optim, Tensor

from torch_sparse import SparseTensor, matmul

from torch_geometric.utils import structured_negative_sampling
from torch_geometric.data import download_url, extract_zip
from torch_geometric.nn.conv.gcn_conv import gcn_norm
from torch_geometric.nn.conv import MessagePassing
from torch_geometric.typing import Adj

# Link Prediction on MovieLens

This colab notebook shows how to load a set of `*.csv` files as input and construct a heterogeneous graph from it.
We will then use this dataset as input into a [heterogeneous graph model](https://pytorch-geometric.readthedocs.io/en/latest/notes/heterogeneous.html#hgtutorial), and use it for the task of link prediction.
A few code cells require user input to let the code run through successfully.
Parts of this tutorial are also available in [our documentation](https://pytorch-geometric.readthedocs.io/en/latest/notes/load_csv.html).

We are going to use the [MovieLens dataset](https://grouplens.org/datasets/movielens/) collected by the GroupLens research group.
This toy dataset describes ratings and tagging activity from MovieLens.
The dataset contains approximately 100k ratings across more than 9k movies from more than 600 users.
We are going to use this dataset to generate two node types holding data for movies and users, respectively, and one edge type connecting users and movies, representing the relation of whether a user has rated a specific movie.

The link prediction task then tries to predict missing ratings, and can, for example, be used to recommend users new movies.

## Heterogeneous Graph Creation

First, we download the dataset to an arbitrary folder (in this case, the current directory):

In [3]:
from torch_geometric.data import download_url, extract_zip

url = 'https://files.grouplens.org/datasets/movielens/ml-latest-small.zip'
extract_zip(download_url(url, '.'), '.')

movies_path = './ml-latest-small/movies.csv'
ratings_path = './ml-latest-small/ratings.csv'

Extracting ./ml-latest-small.zip


Before we create the heterogeneous graph, let’s take a look at the data.

In [60]:
import pandas as pd

print('movies.csv:')
print('===========')
print(pd.read_csv(movies_path)[["movieId", "genres"]].head())
print()
print('ratings.csv:')
print('============')
print(pd.read_csv(ratings_path)[["userId", "movieId"]].head())

movies.csv:
   movieId                                       genres
0        1  Adventure|Animation|Children|Comedy|Fantasy
1        2                   Adventure|Children|Fantasy
2        3                               Comedy|Romance
3        4                         Comedy|Drama|Romance
4        5                                       Comedy

ratings.csv:
   userId  movieId
0       1        1
1       1        3
2       1        6
3       1       47
4       1       50


We see that the `movies.csv` file provides two useful columns: `movieId` assigns a unique identifier to each movie, while the `genres` column represent genres of the given movie.
We can make use of this column to define a feature representation that can be easily interpreted by machine learning models.

In [61]:
# Load the entire movie data frame into memory:
movies_df = pd.read_csv(movies_path, index_col='movieId')

# Split genres and convert into indicator variables:
genres = movies_df['genres'].str.get_dummies('|')
print(genres[["Action", "Adventure", "Drama", "Horror"]].head())

# Use genres as movie input features:
movie_feat = torch.from_numpy(genres.values).to(torch.float)
assert movie_feat.size() == (9742, 20)  # 20 genres in total.

         Action  Adventure  Drama  Horror
movieId                                  
1             0          1      0       0
2             0          1      0       0
3             0          0      0       0
4             0          0      1       0
5             0          0      0       0


In [88]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
movie_feat = movie_feat.to(device)

In [63]:
movie_feat.shape

torch.Size([9742, 20])

The `ratings.csv` data connects users (as given by `userId`) and movies (as given by `movieId`).
Due to simplicity, we do not make use of the additional `timestamp` and `rating` information.
Here, we first read the `*.csv` file from disk, and create a mapping that maps entry IDs to a consecutive value in the range `{ 0, ..., num_rows - 1 }`.
This is needed as we want our final data representation to be as compact as possible, *e.g.*, the representation of a movie in the first row should be accessible via `x[0]`.

Afterwards, we obtain the final `edge_index` representation of shape `[2, num_ratings]` from `ratings.csv` by merging mapped user and movie indices with the raw indices given by the original data frame.

In [64]:
# Load the entire ratings data frame into memory:
ratings_df = pd.read_csv(ratings_path)
ratings_df = ratings_df[ratings_df['rating'] >= 3]

# Create a mapping from unique user indices to range [0, num_user_nodes):
unique_user_id = ratings_df['userId'].unique()
unique_user_id = pd.DataFrame(data={
    'userId': unique_user_id,
    'mappedID': pd.RangeIndex(len(unique_user_id)),
})
print("Mapping of user IDs to consecutive values:")
print("==========================================")
print(unique_user_id.head())
print()
# Create a mapping from unique movie indices to range [0, num_movie_nodes):
unique_movie_id = pd.DataFrame(data={
    'movieId': movies_df.index,
    'mappedID': pd.RangeIndex(len(movies_df)),
})
print("Mapping of movie IDs to consecutive values:")
print("===========================================")
print(unique_movie_id.head())

# Perform merge to obtain the edges from users and movies:
ratings_user_id = pd.merge(ratings_df['userId'], unique_user_id,
                            left_on='userId', right_on='userId', how='left')
ratings_user_id = torch.from_numpy(ratings_user_id['mappedID'].values)
ratings_movie_id = pd.merge(ratings_df['movieId'], unique_movie_id,
                            left_on='movieId', right_on='movieId', how='left')
ratings_movie_id = torch.from_numpy(ratings_movie_id['mappedID'].values)

# With this, we are ready to construct our `edge_index` in COO format
# following PyG semantics:
edge_index = torch.stack([ratings_user_id, ratings_movie_id], dim=0)


print()
print("Final edge indices pointing from users to movies:")
print("=================================================")
print(edge_index)

Mapping of user IDs to consecutive values:
   userId  mappedID
0       1         0
1       2         1
2       3         2
3       4         3
4       5         4

Mapping of movie IDs to consecutive values:
   movieId  mappedID
0        1         0
1        2         1
2        3         2
3        4         3
4        5         4

Final edge indices pointing from users to movies:
tensor([[   0,    0,    0,  ...,  608,  608,  608],
        [   0,    2,    5,  ..., 9462, 9463, 9503]])


In [65]:
num_users = len(unique_user_id)
num_movies = len(unique_movie_id)

### Train-test split

In [66]:
# split the edges of the graph using a 80/10/10 train/validation/test split
num_interactions = edge_index.shape[1]
all_indices = [i for i in range(num_interactions)]

train_indices, test_indices = train_test_split(
    all_indices, test_size=0.2, random_state=42)
val_indices, test_indices = train_test_split(
    test_indices, test_size=0.5, random_state=42)

train_edge_index = edge_index[:, train_indices]
val_edge_index = edge_index[:, val_indices]
test_edge_index = edge_index[:, test_indices]

In [67]:
# convert edge indices into Sparse Tensors: https://pytorch-geometric.readthedocs.io/en/latest/notes/sparse_tensor.html
train_sparse_edge_index = SparseTensor(row=train_edge_index[0], col=train_edge_index[1], sparse_sizes=(
    num_users + num_movies, num_users + num_movies))
val_sparse_edge_index = SparseTensor(row=val_edge_index[0], col=val_edge_index[1], sparse_sizes=(
    num_users + num_movies, num_users + num_movies))
test_sparse_edge_index = SparseTensor(row=test_edge_index[0], col=test_edge_index[1], sparse_sizes=(
    num_users + num_movies, num_users + num_movies))

In [68]:
# function which random samples a mini-batch of positive and negative samples
def sample_mini_batch(batch_size, edge_index):
    """Randomly samples indices of a minibatch given an adjacency matrix

    Args:
        batch_size (int): minibatch size
        edge_index (torch.Tensor): 2 by N list of edges

    Returns:
        tuple: user indices, positive item indices, negative item indices
    """
    edges = structured_negative_sampling(edge_index)
    edges = torch.stack(edges, dim=0)
    indices = random.choices(
        [i for i in range(edges[0].shape[0])], k=batch_size)
    batch = edges[:, indices]
    user_indices, pos_item_indices, neg_item_indices = batch[0], batch[1], batch[2]
    return user_indices, pos_item_indices, neg_item_indices

In [69]:
'''
Samples a negative edge :obj:`(i,k)` for every positive edge :obj:`(i,j)` in the 
graph given by :attr:`edge_index`, and returns it as a tuple of the form :obj:`(i,j,k)`.
'''
edges = structured_negative_sampling(train_edge_index)
edges = torch.stack(edges, dim=0)
edges

tensor([[ 437,  446,  451,  ...,  597,    6,  121],
        [5161,  508, 3640,  ..., 6573, 5687, 3216],
        [ 155, 9374, 5577,  ...,  476, 7368, 3020]])

## LightGCN layer

In [91]:
from torch_geometric.nn.conv.gcn_conv import gcn_norm
from torch import nn
import torch.nn.functional as F
from torch_geometric.nn.conv import MessagePassing


class LightGCN(MessagePassing):
  def __init__(self, num_users, num_movies, hidden_channels, movie_feat, num_layers = 3, add_self_loops = False):
    super().__init__()
    self.num_users = num_users
    self.num_movies = num_movies
    self.hidden_channels = hidden_channels
    self.num_layers = num_layers
    self.add_self_loops = add_self_loops
    self.movie_feat = movie_feat
    self.movie_feat = self.movie_feat.to(device)

    self.movie_lin = nn.Linear(20, self.hidden_channels)
    self.users_emb = nn.Embedding(num_embeddings=self.num_users, embedding_dim=self.hidden_channels) # e_u^0
    self.items_emb = nn.Embedding(num_embeddings=self.num_movies, embedding_dim=self.hidden_channels) # e_i^0

    nn.init.normal_(self.users_emb.weight, std=0.1)
    nn.init.normal_(self.items_emb.weight, std=0.1)

    self.users_emb = self.users_emb.to(device)
    self.items_emb = self.items_emb.to(device)

  def forward(self, edge_index):
    edge_index_norm = gcn_norm(edge_index, add_self_loops = self.add_self_loops)

    #print(self.users_emb.weight.get_device())

    print(self.movie_lin(self.movie_feat).get_device())
    
    emb_0 = torch.cat([self.users_emb.weight, self.items_emb.weight + self.movie_lin(self.movie_feat)]) # E^0
    embs = [emb_0]
    emb_k = emb_0

    # multi-scale diffusion
    for i in range(self.num_layers):
      emb_k = self.propagate(edge_index_norm, x=emb_k)
      embs.append(emb_k)

    embs = torch.stack(embs, dim=1)
    emb_final = torch.mean(embs, dim=1) # E^K

    users_emb_final, items_emb_final = torch.split(emb_final, [self.num_users, self.num_items]) # splits into e_u^K and e_i^K

    # returns e_u^K, e_u^0, e_i^K, e_i^0
    return users_emb_final, self.users_emb.weight, items_emb_final, self.items_emb.weight + self.movie_lin(movie_feat)

  def message(self, x_j: Tensor) -> Tensor:
    return x_j
  
  def message_and_aggregate(self, adj_t: SparseTensor, x: Tensor) -> Tensor:
    # computes \tilde{A} @ x
    return matmul(adj_t, x)


# Our final classifier applies the dot-product between source and destination
# node embeddings to derive edge-level predictions:
class Classifier(torch.nn.Module):
    def forward(self, x_user: Tensor, x_movie: Tensor, edge_label_index: Tensor) -> Tensor:
        # Convert node embeddings to edge-level representations:
        edge_feat_user = x_user[edge_label_index[0]]
        edge_feat_movie = x_movie[edge_label_index[1]]

        # Apply dot-product to get a prediction per supervision edge:
        return (edge_feat_user * edge_feat_movie).sum(dim=-1)


class Model(torch.nn.Module):
    def __init__(self, hidden_channels):
        super().__init__()
        # Since the dataset does not come with rich features, we also learn two
        # embedding matrices for users and movies:
        '''
        self.movie_lin = torch.nn.Linear(20, hidden_channels)
        self.user_emb = torch.nn.Embedding(num_users, hidden_channels)
        self.movie_emb = torch.nn.Embedding(num_movies, hidden_channels)
        '''

        self.gcn = LightGCN(num_users, num_movies, hidden_channels, movie_feat)

        self.classifier = Classifier()

    def forward(self, edge_index) -> Tensor:
        # `x_dict` holds feature matrices of all node types
        # `edge_index_dict` holds all edge indices of all edge types

        users_emb_final, users_emb_0, items_emb_final, items_emb_0 = self.gcn.forward(edge_index)

        pred = self.classifier(
            users_emb_final,
            items_emb_final,
            edge_index,
        )

        return pred

        
model = Model(hidden_channels=64)

print(model)

Model(
  (gcn): LightGCN()
  (classifier): Classifier()
)


In [75]:
def bpr_loss(users_emb_final, users_emb_0, pos_items_emb_final, pos_items_emb_0, neg_items_emb_final, neg_items_emb_0, lambda_val):
    """Bayesian Personalized Ranking Loss as described in https://arxiv.org/abs/1205.2618

    Args:
        users_emb_final (torch.Tensor): e_u_k
        users_emb_0 (torch.Tensor): e_u_0
        pos_items_emb_final (torch.Tensor): positive e_i_k
        pos_items_emb_0 (torch.Tensor): positive e_i_0
        neg_items_emb_final (torch.Tensor): negative e_i_k
        neg_items_emb_0 (torch.Tensor): negative e_i_0
        lambda_val (float): lambda value for regularization loss term

    Returns:
        torch.Tensor: scalar bpr loss value
    """
    reg_loss = lambda_val * (users_emb_0.norm(2).pow(2) +
                             pos_items_emb_0.norm(2).pow(2) +
                             neg_items_emb_0.norm(2).pow(2)) # L2 loss

    pos_scores = torch.mul(users_emb_final, pos_items_emb_final)
    pos_scores = torch.sum(pos_scores, dim=-1) # predicted scores of positive samples
    neg_scores = torch.mul(users_emb_final, neg_items_emb_final)
    neg_scores = torch.sum(neg_scores, dim=-1) # predicted scores of negative samples

    loss = -torch.mean(torch.nn.functional.softplus(pos_scores - neg_scores)) + reg_loss

    return loss

In [76]:
model = model.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [92]:
# training loop
LAMBDA = 1E-6
ITERATION = 10
BATCH_SIZE = 1024
train_losses = []
val_losses = []

for iter in range(ITERATION):
    # forward propagation
    total_loss = total_examples = 0

    users_emb_final, users_emb_0, items_emb_final, items_emb_0 = model.forward(
        train_sparse_edge_index)

    # mini batching
    user_indices, pos_item_indices, neg_item_indices = sample_mini_batch(
        BATCH_SIZE, train_edge_index)
    user_indices, pos_item_indices, neg_item_indices, movie_feat = user_indices.to(
        device), pos_item_indices.to(device), neg_item_indices.to(device), movie_feat.to(device)
    users_emb_final, users_emb_0 = users_emb_final[user_indices], users_emb_0[user_indices]
    pos_items_emb_final, pos_items_emb_0 = items_emb_final[
        pos_item_indices], items_emb_0[pos_item_indices]
    neg_items_emb_final, neg_items_emb_0 = items_emb_final[
        neg_item_indices], items_emb_0[neg_item_indices]

    # loss computation
    train_loss = bpr_loss(users_emb_final, users_emb_0, pos_items_emb_final,
                          pos_items_emb_0, neg_items_emb_final, neg_items_emb_0, LAMBDA)

    optimizer.zero_grad()
    train_loss.backward()
    optimizer.step()


    total_loss += float(train_loss) * BATCH_SIZE
    total_examples += BATCH_SIZE

print(f"Epoch: {iter:03d}, Loss: {total_loss / total_examples:.4f}")

RuntimeError: ignored

## Training a Heterogeneous Link-level GNN

Training our GNN is then similar to training any PyTorch model.
We move the model to the desired device, and initialize an optimizer that takes care of adjusting model parameters via stochastic gradient descent.

The training loop then iterates over our mini-batches, applies the forward computation of the model, computes the loss from ground-truth labels and obtained predictions (here we make use of binary cross entropy), and adjusts model parameters via back-propagation and stochastic gradient descent.

In [ ]:
import tqdm

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Device: '{device}'")

model = model.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

for epoch in range(1, 6):
    total_loss = total_examples = 0
    for sampled_data in tqdm.tqdm(train_loader):
        optimizer.zero_grad()

        # TODO: Move `sampled_data` to the respective `device`
        # TODO: Run `forward` pass of the model
        # TODO: Apply binary cross entropy via
        # `F.binary_cross_entropy_with_logits(pred, ground_truth)`
        sampled_data = sampled_data.to(device)
        pred = model.forward(sampled_data)
        loss = F.binary_cross_entropy_with_logits(pred, sampled_data["user", "rates", "movie"].edge_label)


        loss.backward()
        optimizer.step()
        total_loss += float(loss) * pred.numel()
        total_examples += pred.numel()
    print(f"Epoch: {epoch:03d}, Loss: {total_loss / total_examples:.4f}")

Device: 'cuda'


100%|██████████| 154/154 [00:00<00:00, 155.35it/s]


Epoch: 001, Loss: 0.6931


100%|██████████| 154/154 [00:00<00:00, 167.17it/s]


Epoch: 002, Loss: 0.6928


...

Epoch: 003, Loss: 0.6908


100%|██████████| 154/154 [00:00<00:00, 171.04it/s]


Epoch: 004, Loss: 0.6823


100%|██████████| 154/154 [00:00<00:00, 161.29it/s]

Epoch: 005, Loss: 0.6631


## Evaluating a Heterogeneous Link-level GNN

After training, we evaluate our model on useen data coming from the validation set.
For this, we define a new `LinkNeighborLoader` (which now iterates over the edges in the validation set), obtain the predictions on validation edges by running the model, and finally evaluate the performance of the model by computing the AUC score over the set of predictions and their corresponding ground-truth edges (including both positive and negative edges).

In [ ]:
# Define the validation seed edges:
edge_label_index = val_data["user", "rates", "movie"].edge_label_index
edge_label = val_data["user", "rates", "movie"].edge_label

val_loader = LinkNeighborLoader(
    data=val_data,
    num_neighbors=[20, 10],
    edge_label_index=(("user", "rates", "movie"), edge_label_index),
    edge_label=edge_label,
    batch_size=3 * 128,
    shuffle=False,
)

sampled_data = next(iter(val_loader))

print("Sampled mini-batch:")
print("===================")
print(sampled_data)

assert sampled_data["user", "rates", "movie"].edge_label_index.size(1) == 3 * 128
assert sampled_data["user", "rates", "movie"].edge_label.min() >= 0
assert sampled_data["user", "rates", "movie"].edge_label.max() <= 1

Sampled mini-batch:
HeteroData(
  user={
    node_id=[608],
    n_id=[608]
  },
  movie={
    node_id=[2575],
    x=[2575, 20],
    n_id=[2575]
  },
  (user, rates, movie)={
    edge_index=[2, 18358],
    edge_label=[384],
    edge_label_index=[2, 384],
    e_id=[18358],
    input_id=[384]
  },
  (movie, rev_rates, user)={
    edge_index=[2, 7790],
    e_id=[7790]
  }
)


In [ ]:
from sklearn.metrics import roc_auc_score

preds = []
ground_truths = []
for sampled_data in tqdm.tqdm(val_loader):
    with torch.no_grad():
        # TODO: Collect predictions and ground-truths and write them into
        # `preds` and `ground_truths`.
        sampled_data = sampled_data.to(device)
        pred = model.forward(sampled_data)
        preds.append(pred)
        ground_truth = sampled_data["user", "rates", "movie"].edge_label
        ground_truths.append(ground_truth)

pred = torch.cat(preds, dim=0).cpu().numpy()
ground_truth = torch.cat(ground_truths, dim=0).cpu().numpy()
auc = roc_auc_score(ground_truth, pred)
print()
print(f"Validation AUC: {auc:.4f}")

100%|██████████| 64/64 [00:00<00:00, 215.75it/s]



Validation AUC: 0.4258
